In [ ]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense,Flatten

from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import warnings
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 10,
                                   zoom_range = 0.25,
                                   horizontal_flip = True,
                                   vertical_flip=True,
                                   width_shift_range = 0.2,
                                   rotation_range=20,
                                   fill_mode = 'nearest')
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/ML_miniproject/archive (1)/chest_xray (1)/train (1)',
                                                 target_size = (224, 224),
                                                 color_mode="rgb",
                                                 batch_size = 32,
                                                 class_mode = 'binary')

test_datagen = ImageDataGenerator(rescale = 1/255)
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/ML_miniproject/archive (1)/chest_xray (1)/test (1)',
                                            target_size = (224, 224),
                                            color_mode="rgb",
                                            batch_size = 32,
                                            class_mode = 'binary')

val_datagen = ImageDataGenerator(rescale = 1/255)
val_set = test_datagen.flow_from_directory('/content/drive/MyDrive/ML_miniproject/archive (1)/chest_xray (1)/val (1)',
                                            target_size=(224,224),
                                           color_mode="rgb",
                                           batch_size = 2,
                                           class_mode = 'binary')

Found 5219 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Found 16 images belonging to 2 classes.


In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                  patience=10)

In [ ]:
lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                          patience=8)

In [ ]:
def create_model3():
  model3 = Sequential()

  pretrained_model=tf.keras.applications.vgg16.VGG16(include_top=False,
                    input_shape=(224,224,3),
                    pooling='avg',classes=2,
                    weights='imagenet')
  for layer in pretrained_model.layers:
          layer.trainable=False

  model3.add(pretrained_model)
  model3.add(Flatten())
  model3.add(Dense(512, activation='relu'))
  
  model3.add(Dense(1, activation='sigmoid'))
  model3.compile(optimizer=Adam(lr=0.001),loss='binary_crossentropy',metrics=['accuracy'])

  return model3


In [ ]:
vgg_model_new = create_model3()

58889256/58889256 [==============================] - 0s 0us/step


/usr/local/lib/python3.7/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
vgg_model_new.fit(x = training_set, validation_data=val_set,steps_per_epoch=100,callbacks=[early_stopping,lr], epochs = 7,batch_size=32)

Epoch 1/7
100/100 [==============================] - 1796s 17s/step - loss: 0.3601 - accuracy: 0.8322 - val_loss: 0.5813 - val_accuracy: 0.7500 - lr: 0.0010
Epoch 2/7
100/100 [==============================] - 1695s 17s/step - loss: 0.2750 - accuracy: 0.8859 - val_loss: 0.4409 - val_accuracy: 0.7500 - lr: 0.0010
Epoch 3/7
100/100 [==============================] - 1710s 17s/step - loss: 0.2487 - accuracy: 0.8994 - val_loss: 0.4415 - val_accuracy: 0.7500 - lr: 0.0010
Epoch 4/7
100/100 [==============================] - 1697s 17s/step - loss: 0.2447 - accuracy: 0.8978 - val_loss: 0.4346 - val_accuracy: 0.7500 - lr: 0.0010
Epoch 5/7
100/100 [==============================] - 1723s 17s/step - loss: 0.2180 - accuracy: 0.9084 - val_loss: 0.5050 - val_accuracy: 0.7500 - lr: 0.0010
Epoch 6/7
100/100 [==============================] - 1696s 17s/step - loss: 0.2146 - accuracy: 0.9082 - val_loss: 0.4837 - val_accuracy: 0.7500 - lr: 0.0010
Epoch 7/7
100/100 [==============================] - 1694s

In [ ]:
from keras.callbacks import ModelCheckpoint
mc = ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', save_best_only=True)
mc

In [ ]:
vgg_model_new.save('final_model_vgg_final.h5') 

In [ ]:
import numpy as np
from keras.preprocessing import image
from google.colab import files
from keras.models import load_model
from io import BytesIO
from PIL import Image
model=load_model('final_model_vgg_final.h5')

In [ ]:
score = model.evaluate(training_set)

print("Train Loss: ", score[0])
print("Train Accuracy: ", score[1])

score = model.evaluate(test_set)
print("\nTest loss: ", score[0])
print("Test Accuracy: ", score[1])

164/164 [==============================] - 2846s 17s/step - loss: 0.1908 - accuracy: 0.9268
Train Loss:  0.1907568722963333
Train Accuracy:  0.9268059134483337
20/20 [==============================] - 337s 17s/step - loss: 0.2460 - accuracy: 0.9006

Test loss:  0.24604423344135284
Test Accuracy:  0.9006410241127014
